In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.naive_bayes import GaussianNB
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from scipy.sparse import hstack
import time, os, random, sys
import math
import hyperopt.tpe
import hpsklearn.components
import hpsklearn.demo_support
import warnings
warnings.filterwarnings('ignore')
random.seed(1)

/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/garb47/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


In [338]:
example = pd.read_csv('example.csv')

In [390]:
train_df, test_df = train_test_split(example, train_size = 0.8)
X_train = train_df.drop([ 'Grant.Status'], axis=1)
y_train = train_df['Grant.Status']
X_test = test_df.drop([ 'Grant.Status'], axis=1)
y_test = test_df['Grant.Status']

In [393]:
gbclf = GradientBoostingClassifier(n_estimators=30, max_depth=7, verbose=0,learning_rate=0.001)
gbclf.fit(X_train,y_train)
X_train_leaves = gbclf.apply(X_train)[:,:,0]
(train_rows, cols) = X_train_leaves.shape
gbclf.fit(X_test,y_test)
X_test_leaves = gbclf.apply(X_test)[:,:,0]

In [394]:
lr = LogisticRegression()
lr.fit(X_train_leaves, y_train)
y_pred_gbdtlr1 = lr.predict_proba(X_test_leaves)[:,1]
gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)

gbdt+lr auc 1: 0.35355


In [365]:
print(len(X_train_leaves))
print(len(X_train_leaves[0]))
y_pred_gbdtlr1 = lr.predict_proba(X_test_leaves)[:,1]
gbdtlr_auc1 = roc_auc_score(y_test, y_pred_gbdtlr1)
print('gbdt+lr auc 1: %.5f' % gbdtlr_auc1)

6966
30
gbdt+lr auc 1: 0.57923


In [366]:
gbc=GradientBoostingClassifier()
gbc.fit(X_train,y_train)
y_pred=gbc.predict_proba(X_test)[:,1]
auc=roc_auc_score(y_test,y_pred)
print(auc)

0.948127121275


In [367]:
xgb=XGBClassifier()
xgb.fit(X_train,y_train)
y_pred_xgb=xgb.predict_proba(X_test)[:,1]
auc=roc_auc_score(y_test,y_pred_xgb)
print(auc)

0.950408328783


In [135]:
gnb= GaussianNB()
gnb.fit(X_train_leaves, y_train)
Y_pred_nb=gnb.predict_proba(X_test_leaves)[:,1]
gnb_auc = roc_auc_score(y_test,Y_pred_nb)
print('NB auc: ', gnb_auc)

NB auc:  0.657571872229


In [136]:
svc=SVC(probability=True)

svc.fit(X_train_leaves, y_train)
Y_pred_svc=svc.predict_proba(X_test_leaves)[:,1]
svc_auc=roc_auc_score(y_test,Y_pred_svc)
print('SVC auc: %.5f' % svc_auc)

SVC auc: 0.65157


In [137]:
knn=KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train_leaves, y_train)
Y_pred_knn=knn.predict_proba(X_test_leaves)[:,1]
knn_auc=roc_auc_score(y_test,Y_pred_knn)
print('KNN auc : %.5f' % knn_auc)

KNN auc : 0.80715


In [138]:
#perceptron
perceptron = Perceptron()
perceptron.fit(X_train_leaves, y_train)
Y_pred_prec=perceptron.predict(X_test_leaves)
perc_auc=roc_auc_score(y_test,Y_pred_prec)
print('Perceptron auc : ', perc_auc)

Perceptron auc :  0.633917089256


In [139]:
#linear svc
lin = LinearSVC()
lin.fit(X_train_leaves, y_train)
Y_pred_lin=lin.predict(X_test_leaves)
lin_auc=roc_auc_score(y_test,Y_pred_lin)
print('Linear SVC auc : ', lin_auc)

Linear SVC auc :  0.51237250333


In [140]:
#SGD
sgd = SGDClassifier(loss='log')
sgd.fit(X_train_leaves, y_train)
Y_pred_sgd=sgd.predict_proba(X_test_leaves)[:, 1]
sgd_auc=roc_auc_score(y_test,Y_pred_sgd)
print('SGD auc : %.5f' % sgd_auc)

SGD auc : 0.73946


In [141]:
#XGB

xgb=XGBClassifier()
xgb.fit(X_train_leaves,y_train)
Y_pred_xgb=xgb.predict(X_test_leaves)
xgb_auc= roc_auc_score(y_test,Y_pred_xgb)
print(xgb_auc)

0.633002491391


In [142]:
lr = LogisticRegression(n_jobs=-1)
X_train_ext = hstack([X_train_leaves, X_train])
lr.fit(X_train_ext, y_train)
X_test_ext = hstack([X_test_leaves, X_test])
y_pred_gbdtlr2 = lr.predict_proba(X_test_ext)[:, 1]
gbdtlr_auc2 = roc_auc_score(y_test, y_pred_gbdtlr2)
print('gbdt+lr auc 2: %.5f' % gbdtlr_auc2)

gbdt+lr auc 2: 0.80278


In [360]:
example=pd.read_csv('train_preprocessed2.csv')

In [361]:
example.describe()

,A..papers,A.papers,B.papers,C.papers,Dif.countries,Perc_non_australian,Number.people,PHD,Max.years.univ,Grants.succ,Grants.unsucc,Departments,Perc.Lang.Eng,Perc.Lang.Other,Dif.Role,Perc.CHIEF.INVESTIGATOR,Average.Age,Weekday,Month,Day.of.Month,Day.of.Year,Season,Sponsor.Code,Grant.Category.Code,Contract.Value.Band,RFCD.0,RFCD.1,RFCD.2,RFCD.3,RFCD.4,RFCD.5,RFCD.6,RFCD.7,RFCD.8,RFCD.9,RFCD.10,RFCD.11,RFCD.12,RFCD.13,RFCD.14,RFCD.15,RFCD.16,RFCD.17,RFCD.18,RFCD.19,RFCD.20,RFCD.21,RFCD.22,RFCD.23,RFCD.24,SEO.0,SEO.1,SEO.2,SEO.3,SEO.4,SEO.5,SEO.6,SEO.7,SEO.8,SEO.9,SEO.10,SEO.11,SEO.12,SEO.13,SEO.14,SEO.15,SEO.16,SEO.17,SEO.18,SEO.19,Grant.Status
count,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8.708000e+03,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.00000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000,8708.000000
mean,5.165136,7.189596,5.592444,3.017800,1.048576,0.659637,1.495981,0.917432,11.490583,1.731626,3.061667,1.070395,0.103966,0.026145,1.359784,0.757441,55.278975,2.735645,7.347152,15.047772,207.657556,2.833142,36.816146,3.043523,2.507464e+05,3.306040,0.001952,0.030776,0.045935,0.053973,0.035599,0.262862,0.065572,0.083142,0.089228,0.019982,0.670648,0.041915,0.021704,0.014929,0.019522,0.044442,0.046164,0.028020,0.011024,0.022393,0.037667,0.029398,0.015158,0.001952,3.204180,0.008957,0.033877,0.023542,0.004708,0.004249,0.012632,0.108980,0.02802,0.004593,0.046394,0.008842,0.030087,0.974736,0.047542,0.189022,0.020211,0.099104,0.148254,0.002067,0.458429
std,11.967629,12.348254,8.592110,5.940251,0.543972,0.444329,1.040738,0.897762,11.651806,2.326951,4.491482,0.492256,0.303809,0.156832,0.570493,0.335736,8.617908,1.932758,3.458221,8.215053,108.696075,1.088512,66.352391,2.831441,2.868332e+06,1.006662,0.044143,0.225754,0.295370,0.314759,0.265761,0.635942,0.345534,0.369972,0.385160,0.185163,0.837382,0.278987,0.201891,0.141389,0.186448,0.264643,0.268443,0.244184,0.122632,0.173014,0.250793,0.198326,0.165331,0.044143,1.050082,0.098979,0.220934,0.188742,0.079338,0.074897,0.130644,0.383053,0.22914,0.080066,0.258819,0.094842,0.210584,1.090193,0.273713,0.612911,0.176890,0.427260,0.399207,0.045421,0.498297
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,5.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.500000,49.500000,1.000000,4.000000,8.000000,121.000000,2.000000,2.000000,1.000000,0.000000e+00,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [348]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1500)

In [352]:
example.loc[:, example.isnull().any()]

,Average.Age
0,52.000000
1,57.000000
2,65.750000
3,52.000000
4,52.000000
5,67.000000
6,60.333333
7,64.500000
8,69.500000
9,87.000000


In [353]:
example=example.fillna(0)

In [368]:
len(example.columns)

71